In [ ]:
!pip install tf-nightly

     |████████████████████████████████| 410.0MB 28kB/s 
     |████████████████████████████████| 6.1MB 46.4MB/s 
     |████████████████████████████████| 4.0MB 39.8MB/s 
     |████████████████████████████████| 4.0MB 45.4MB/s 
     |████████████████████████████████| 471kB 37.9MB/s 
     |████████████████████████████████| 3.8MB 41.5MB/s 
ERROR: tensorflow 2.4.1 has requirement gast==0.3.3, but you'll have gast 0.4.0 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement grpcio~=1.32.0, but you'll have grpcio 1.34.1 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement h5py~=2.10.0, but you'll have h5py 3.1.0 which is incompatible.
  Found existing installation: grpcio 1.32.0
    Uninstalling grpcio-1.32.0:
      Successfully uninstalled grpcio-1.32.0
  Found existing installation: h5py 2.10.0
    Uninstalling h5py-2.10.0:
      Successfully uninstalled h5py-2.10.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os

author_dir = '/content/gdrive/MyDrive/캐글스터디/소설작가분류'
train_dir = os.path.join(author_dir, 'train')

In [ ]:
import pandas as pd
import numpy as np
train =pd.read_csv('/content/gdrive/MyDrive/캐글스터디/소설작가분류/train.csv')
test = pd.read_csv('/content/gdrive/MyDrive/캐글스터디/소설작가분류/test_x.csv')

#Data Preprocessing

1. 데이터를 눈으로 확인하며 특수문자, 불용어, 문장구조에 대한 감을 잡는다.

2. 불용어를 설정하고 리스트에 저장한다. 

3. 불용어 이외의 특수 문자들을 제거한다.(정규표현식 패키지(re)를 사용)

4. 형태소 분석을 통해 문장을 형태소 단위의 토큰으로 분리.이때 내가 설정한 불용어들을 결과로 반환해주는 형태소 분석기를 사용 해야함. 예를 들어 조사를 불용어로 설정하였는데 조사를 분리해주지 못하는 형태소 분석기는 후보에서 제외하면 됨.

5. 형태소 단위의 토큰들을 기반으로 리스트에 저장된 불용어 제거

In [ ]:
train

,index,text,author
0,0,"He was almost choking. There was so much, so m...",3
1,1,"“Your sister asked for it, I suppose?”",2
2,2,"She was engaged one day as she walked, in per...",1
3,3,"The captain was in the porch, keeping himself ...",4
4,4,"“Have mercy, gentlemen!” odin flung up his han...",3
...,...,...,...
54874,54874,"“Is that you, Mr. Smith?” odin whispered. “I h...",2
54875,54875,"I told my plan to the captain, and between us ...",4
54876,54876,"""Your sincere well-wisher, friend, and sister...",1
54877,54877,“Then you wanted me to lend you money?”,3


In [ ]:
train['author'].unique()

array([3, 2, 1, 4, 0])

In [ ]:
X = train.drop('author', axis = 1)
X = X.drop(labels='index',axis = 1)
X

,text
0,"He was almost choking. There was so much, so m..."
1,"“Your sister asked for it, I suppose?”"
2,"She was engaged one day as she walked, in per..."
3,"The captain was in the porch, keeping himself ..."
4,"“Have mercy, gentlemen!” odin flung up his han..."
...,...
54874,"“Is that you, Mr. Smith?” odin whispered. “I h..."
54875,"I told my plan to the captain, and between us ..."
54876,"""Your sincere well-wisher, friend, and sister..."
54877,“Then you wanted me to lend you money?”


In [ ]:
y = train.drop(labels='index',axis = 1)
y = y.drop(labels='text',axis = 1)
y

,author
0,3
1,2
2,1
3,4
4,3
...,...
54874,2
54875,4
54876,1
54877,3


#데이터 토큰화

In [ ]:
from keras.preprocessing.text  import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen = 100 # 100개 이후 단어는 버림
max_words = 107402

token = Tokenizer()
token.fit_on_texts(X['text'])
sequences = token.texts_to_sequences(X['text'])

word_index = token.word_index
print('%s개의 고유한 토큰을 찾았습니다.' %len(word_index))

data = pad_sequences(sequences,maxlen = maxlen)
labels = np.asarray(y)
print('데이터 텐서의 크기:', data.shape)
print('레이블 텐서의 크기:', labels.shape)


42330개의 고유한 토큰을 찾았습니다.
데이터 텐서의 크기: (54879, 100)
레이블 텐서의 크기: (54879, 1)


In [ ]:
from tensorflow.keras.utils import to_categorical
y= to_categorical(labels)
y

array([[0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       ...,
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.]], dtype=float32)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test =train_test_split(data, y, test_size=0.2, random_state=42)

#GloVe 단어 임베딩 내려받기


In [ ]:
glove_dir = '/content/drive/MyDrive/캐글스터디/GloVe/'
embeddings_index = {}
f = open(os.path.join('/content/gdrive/MyDrive/캐글스터디/GloVe/glove.6B.300d.txt'), encoding='utf-8')
for line in f:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:], dtype = 'float32')
  embeddings_index[word] = coefs
f.close()

print('%s개의 단어를 찾았습니다.' %len(embeddings_index))

107402개의 단어를 찾았습니다.


In [ ]:

'''embedding_matrix = np.zeros((len(embeddings_index), 300)) #300차원의 임베딩 매트릭스 생성

for index, word in enumerate(embeddings_index): #vocabulary에 있는 토큰들을 하나씩 넘겨줍니다.
    if word in word_index: #넘겨 받은 토큰이 word2vec에 존재하면(이미 훈련이 된 토큰이라는 뜻)
        embedding_vector = embeddings_index[word] #해당 토큰에 해당하는 vector를 불러오고
        embedding_matrix[i] = embedding_vector #해당 위치의 embedding_mxtrix에 저장합니다.
    else:
        print("[0]glove 없는 단어입니다.".format(index))
        break
        '''

In [ ]:
embedding_dim = 300
embedding_matrix= np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
  if i < max_words:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
      embedding_matrix[i] = embedding_vector

In [ ]:
embedding_matrix.shape

(107402, 300)

#Model


In [ ]:
from keras.models  import Sequential
from keras.layers  import Embedding, Flatten, Dense,MaxPooling1D
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length= 100))  #https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding
                                                                      # Embedding layer에서는 Embedding(input_dim, output_dim을 기본으로 주입)
                                                                      #Embedding 층은 크기가 (samples, sequence_length)인 2D 정수텐서를 입력으로 받는다.
                                                                      # 배치에 있는 모든 시퀀스의 크기는 같아야 하므로 작은 길이의 시퀀스는 0으로 패딩되고 길이가 더 긴 시퀀스는 잘린다.
                                                                      #Embedding 층은  크기가 (sampels, sequence_length, embedding_dimentionality)인 3D 실수형 텐서를 반환한다.
                                                                      # 이런 3D텐서는 RNN층이나 1D합성곱 층에서 처리된다.

model.add(Flatten())
model.add(Dense(5000, activation = 'relu'))
model.add(Dense(1000, activation = 'relu'))
model.add(Dense(200, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(5, activation = 'softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 300)          32220600  
_________________________________________________________________
flatten_1 (Flatten)          (None, 30000)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 5000)              150005000 
_________________________________________________________________
dense_7 (Dense)              (None, 1000)              5001000   
_________________________________________________________________
dense_8 (Dense)              (None, 200)               200200    
_________________________________________________________________
dense_9 (Dense)              (None, 64)                12864     
_________________________________________________________________
dense_10 (Dense)             (None, 5)                

# pretrainned embedding 사용시 주의점

**Embedding을 로드 한 후 동결한다.** 훈련하는 동안 pre-trainned된 부분이 업데이트 된다면 이미 알고 있던 정보를 모두 잃게 됨.랜덤하게 초기화된 층에서 대량의 그래디언트 업데이트가 발생하면 이미 학습된 특성을 오염시키기 때문




In [ ]:
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False  # Embedding을 로드 한 후 동결한다. 훈련하는 동안 pre-trainned된 부분이 업데이트 된다면 이미 알고 있던 정보를 모두 잃게 됨.
                                   # 랜덤하게 초기화된 층에서 대량의 그래디언트 업데이트가 발생하면 이미 학습된 특성을 오염시키기 때문



#Train model & evaluate

맨처음에 model이 실행이 안되서 고생좀 했는데 .맨 처음에 데이터 토큰화 부분에서 max_words = 10000으로 (가장 빈도수가 높은 10000개의 단어만 사용) 설정해놔서 train_data중에 10000개 안에 못드는 단어를 찾으면 바로 트레이닝이 중지됨.
그래서 max_words를 300차원 임베딩 공간에 있는 107402개를 모두 사용했음. 근데 토큰화 한걸 보면 42330개의 토큰 밖에 없는 걸 보면 굉장히 비효율 적인것을 알수 있음.
다른 방법이 필요할듯?

In [ ]:
model.compile(optimizer='Adam',
              loss = 'CategoricalCrossentropy',
              metrics = ['accuracy'])
history = model.fit (X_train, y_train, validation_split=0.2, epochs = 10, batch_size = 8, verbose = 2)

Epoch 1/10
